# Topic Modelling

#### Evan Yathon

This notebook is intended to be run with papermill from the project root.

The purpose of this notebook is to process the review title and content to extract key phrases in each review.  Then key phrases will be used in a regression analysis to find out what is most important for a reviewer in recommending or not recommending the airline.

Topic modelling using LDA will be the tool of choice.

Usage:

`papermill src/ipynbs/topic_modelling.ipynb -p load_path data/cleaned_gw_reviews.csv`

In [1]:
#parameters section for Papermill

load_path = "../../data/cleaned_gw_reviews.csv"

In [45]:
# loading packages

# utils
import pandas as pd
import re
import numpy as np

# NLP
import spacy
import gensim 
import gensim.corpora as corpora
from gensim.corpora import Dictionary
from gensim import models
import pyLDAvis.gensim

# display preference for ipynbs
%matplotlib inline

In [3]:
# ensure the dates are parsed correctly with parse_dates argument
reviews = pd.read_csv(load_path, parse_dates = ["date_of_review", "date_flown"])

In [4]:
reviews.head()

,title,review_value,n_user_reviews,reviewer_name,reviewer_country,date_of_review,review_text,aircraft,traveller_type,seat_type,route,date_flown,seat_comfort_rating,cabin_staff_service_rating,food_and_beverages_rating,inflight_entertainment_rating,ground_service_rating,value_for_money_rating,recommendation
0,"""Seat was fine with enough legroom""",7.0,8 reviews,Sander van Kan,Netherlands,2019-07-01,✅ Trip Verified | Dusseldorf to Berlin. Eurowi...,A319,Couple Leisure,Economy Class,Dusseldorf to Berlin,2019-06-01,4.0,3.0,NaN,1.0,3.0,5.0,yes
1,"""crew were smiling and good""",6.0,8 reviews,Sander van Kan,Netherlands,2019-07-01,✅ Trip Verified | Berlin to Dusseldorf. Eurowi...,A319,Couple Leisure,Economy Class,Berlin to Dusseldorf,2019-06-01,3.0,3.0,NaN,1.0,3.0,5.0,yes
2,"""only two agents available""",1.0,6 reviews,Andrew Maynard,United Kingdom,2017-01-04,Check in process at Cologne very poor. Flight ...,NaN,Couple Leisure,Economy Class,CGN to MAN,2017-01-01,2.0,2.0,NaN,NaN,1.0,2.0,no
3,"""good flight and friendly staff""",7.0,1 reviews,T Steen,Netherlands,2016-09-13,✅ Verified Review | Amsterdam to Stuttgart. G...,NaN,Business,Economy Class,AMS to STR,2016-09-01,5.0,5.0,1.0,NaN,5.0,5.0,yes
4,"""never been treated as badly""",1.0,NaN,Karen Kirner,Austria,2016-08-16,✅ Verified Review | I have been a frequent tr...,NaN,Business,Economy Class,DUS to VIE,2016-08-01,1.0,1.0,NaN,NaN,3.0,1.0,no


### Topic Modelling - Preprocessing

Prior to LDA, preprocessing of the text is necessary.  We need to:
- Tokenize each document (review)
- Minimum token length
- Remove stopwords
- Lemmatize
- Consider only specific parts of speech (nouns, verbs etc.)

Use [`spaCy`](https://spacy.io/) for this.

In [5]:
# load the english spacy model
eng_nlp = spacy.load('en')

In [6]:
# create a function to preprocess the reviews and review titles

def preprocess_text(text, min_token_len = 2, relevant_pos = ["NOUN", "VERB", "ADJ"]):
    """
    Given text, min_token_len, and relevant_pos carry out preprocessing of the text 
    and return a preprocessed string. 
    
    Keyword arguments:
    text (str): the text to be preprocessed
    min_token_len (int): min_token_length required
    relevant_pos (list): a list of relevant pos tags
    
    Returns: (str) the preprocessed text
    """
    # use several regex expressions to preprocess strange characters
    # or unwanted text
    
    # remove verified emoji
    text = re.sub(r"✅", "", text)
    
    # remove verified review and trip verified
    text = re.sub(r"Verified Review|Trip Verified", "", text)
    
    # remove anything that is not a word
    text = re.sub(r"[^\w]", " ", text)
    
    # replace multiple spaces with a single space
    text = re.sub(r"\s+", " ", text)
    
    # remove numbers
    text = re.sub(r"[0-9]+", "", text)
    
    # change all text to lowercase
    text = text.lower()
    
    # remove common terms found in topic modelling output
    # eg/ germanwings, flight
    # these words are not useful for this application as they
    # are common terms found in most reviews, and won't be indicative of 
    # things that influenced their recommendation
    terms = ["flight", "germanwing", "fly", "review"]
    text = re.sub("|".join(terms)," ", text)
    
    # utilize spaCy to tokenize and break text into lemmas/POS:
    # - remove stopwords
    # - reduce words to their lemmas
    # - only keep relevant parts of speech
    # - remove short words less than min_token_len
    
    # spacy will utilize
    doc = eng_nlp(text)
    
    processed_text = ""
    
    for token in doc:
        if not token.is_stop and len(token.text) >= min_token_len:# and token.pos_ in relevant_pos:
            processed_text += " " + token.lemma_
    
    return processed_text
    

Use the preprocessing function to process all review texts and review titles.

A good check will be to see if the review topics somewhat make sense with the title topics

In [7]:
reviews["clean_review_text"] = reviews["review_text"].apply(preprocess_text, min_token_len = 3)
reviews["clean_title"] = reviews["title"].apply(preprocess_text, min_token_len = 3)

In [8]:
reviews[["clean_review_text", "review_text", "clean_title", "title"]].head(10)

,clean_review_text,review_text,clean_title,title
0,dusseldorf berlin eurowing operate slight del...,✅ Trip Verified | Dusseldorf to Berlin. Eurowi...,seat fine legroom,"""Seat was fine with enough legroom"""
1,berlin dusseldorf eurowing operate different ...,✅ Trip Verified | Berlin to Dusseldorf. Eurowi...,crew smile good,"""crew were smiling and good"""
2,check process cologne poor display delay minu...,Check in process at Cologne very poor. Flight ...,agent available,"""only two agents available"""
3,amsterdam stuttgart good friendly staff airpl...,✅ Verified Review | Amsterdam to Stuttgart. G...,good friendly staff,"""good flight and friendly staff"""
4,frequent traveler year fly distance cabin cla...,✅ Verified Review | I have been a frequent tr...,treat badly,"""never been treated as badly"""
5,fly london stanste cologne bonn delay hour lo...,✅ Verified Review | Flew Germanwings from Lon...,cramp,"""very cramped"""
6,boyfriend book basic fare hamburg rome fiumic...,My boyfriend and I booked the basic fare from ...,staff friendly,"""staff were friendly"""
7,operation transfer take eurowing read eurowin...,Germanwings operations have now been transferr...,refer eurowing,PLEASE REFER TO EUROWINGS
8,lufthansa operate barcelona berlin tegel figh...,Lufthansa operated by Germanwings from Barcelo...,great value money,"""great value for money"""
9,bangkok london cologne imagine ryanair hour l...,Germanwings from Bangkok to London via Cologne...,pay little money,"""pay very little money"""


It looks mostly good, but I noticed that in the first review it is removing 'enough'.  This could be valuable in context, but I'll keep digging for now.  It's an issue since 'enough' is a stopword and is being removed.

### Topic Modelling: Dictionary and Document-Term Co-occurrence Matrix

In [9]:
# change the cleaned reviews and cleaned titles to 
# an array, required by corpora.Dictionary
review_corpus = [doc.split() for doc in reviews["clean_review_text"].tolist()]
title_corpus = [doc.split() for doc in reviews["clean_title"].tolist()]

In [10]:
# use gensim dictionary to map between words and integer ids
dct_review = corpora.Dictionary(review_corpus)
dct_title = corpora.Dictionary(title_corpus)

In [11]:
# create the document-term co-occurrence matrices
doc_term_mat_review = [dct_review.doc2bow(doc) for doc in review_corpus]
doc_term_mat_title = [dct_title.doc2bow(doc) for doc in title_corpus]

### Topic Modelling:  LDA Topic Model

Number of topics was tuned using the visualizations below by examining the Intertopic distance map.  When there was no overlap and the topics were separated by a fair amount, the number of topics was chosen.

In [35]:
# create the LDA models
# use a lower alpha, meaning that each document is representative of only a few topics
# do this so that each review doesn't contain similar topics and will have variation
# that we can use to figure out why the recommendation was a yes or no

lda_review = models.LdaModel(corpus = doc_term_mat_review, id2word = dct_review, num_topics = 4, alpha = 0.5, passes = 5)
lda_title = models.LdaModel(corpus = doc_term_mat_title, id2word = dct_title, num_topics = 3, alpha = 0.01, passes = 5)

In [36]:
# view the topics and the words that describe them
lda_review.print_topics()

[(0,
  '0.017*"seat" + 0.014*"time" + 0.012*"airline" + 0.010*"check" + 0.009*"pay" + 0.009*"cologne" + 0.009*"return" + 0.009*"cgn" + 0.008*"service" + 0.008*"good"'),
 (1,
  '0.014*"ticket" + 0.012*"lufthansa" + 0.011*"delay" + 0.010*"hour" + 0.009*"check" + 0.009*"price" + 0.009*"seat" + 0.008*"cologne" + 0.007*"book" + 0.007*"berlin"'),
 (2,
  '0.023*"good" + 0.018*"time" + 0.017*"seat" + 0.013*"crew" + 0.012*"friendly" + 0.012*"staff" + 0.011*"check" + 0.011*"cabin" + 0.011*"cologne" + 0.010*"drink"'),
 (3,
  '0.015*"passenger" + 0.012*"delay" + 0.011*"return" + 0.009*"crew" + 0.009*"service" + 0.008*"board" + 0.007*"luggage" + 0.007*"time" + 0.007*"late" + 0.007*"pay"')]

In [37]:
# view the topics and the words that describe them
lda_title.print_topics()

[(0,
  '0.141*"friendly" + 0.100*"staff" + 0.058*"cabin" + 0.058*"crew" + 0.051*"customer" + 0.040*"professional" + 0.038*"delay" + 0.023*"money" + 0.023*"seat" + 0.023*"food"'),
 (1,
  '0.798*"customer" + 0.009*"offer" + 0.009*"airline" + 0.009*"little" + 0.009*"help" + 0.009*"explanation" + 0.009*"choice" + 0.009*"available" + 0.009*"agent" + 0.009*"eurowing"'),
 (2,
  '0.125*"customer" + 0.044*"late" + 0.044*"don" + 0.044*"money" + 0.044*"pay" + 0.044*"little" + 0.044*"good" + 0.044*"crew" + 0.044*"vow" + 0.044*"book"')]

### Topic Modelling: Visualization

In [33]:
# view the reviews 
pyLDAvis.enable_notebook()
vis_reviews = pyLDAvis.gensim.prepare(lda_review, doc_term_mat_review, dct_review, sort_topics=False)
vis_reviews

C:\Users\evany\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.073011  0.002514       1        1  13.107005
1     -0.058925  0.040233       2        1  31.359360
2     -0.025697 -0.071927       3        1  31.085348
3      0.011611  0.029180       4        1  24.448290, topic_info=    Category       Freq         Term      Total  loglift  logprob
43   Default  45.000000    passenger  45.000000  30.0000  30.0000
9    Default  64.000000         good  64.000000  29.0000  29.0000
82   Default  31.000000        cabin  31.000000  28.0000  28.0000
73   Default  27.000000        snack  27.000000  27.0000  27.0000
14   Default  78.000000         seat  78.000000  26.0000  26.0000
614  Default  29.000000          cgn  29.000000  25.0000  25.0000
66   Default  31.000000     friendly  31.000000  24.0000  24.0000
74   Default  42.000000        staff  42.000000  23.0000  23.0000
185  Default  28.000000         tell  28.000000  22.0000  22.0000
41   Default  29.000000      luggage  29.000000  21.0000  21.0000
75   Default  18.000000    stuttgart  18.000000  20.0000  20.0000
133  Default  26.000000          bag  26.000000  19.0000  19.0000
8    Default  24.000000         food  24.000000  18.0000  18.0000
71   Default  16.000000        small  16.000000  17.0000  17.0000
699  Default   6.000000          str   6.000000  16.0000  16.0000
141  Default  19.000000         fare  19.000000  15.0000  15.0000
79   Default  13.000000        water  13.000000  14.0000  14.0000
62   Default   6.000000       bottle   6.000000  13.0000  13.0000
756  Default  14.000000       german  14.000000  12.0000  12.0000
1    Default  23.000000       berlin  23.000000  11.0000  11.0000
559  Default   4.000000       manner   4.000000  10.0000  10.0000
227  Default   6.000000   reasonably   6.000000   9.0000   9.0000
61   Default  25.000000         book  25.000000   8.0000   8.0000
446  Default   8.000000          bus   8.000000   7.0000   7.0000
410  Default   7.000000   definitely   7.000000   6.0000   6.0000
647  Default  12.000000          lhr  12.000000   5.0000   5.0000
743  Default   5.000000     occasion   5.000000   4.0000   4.0000
172  Default  13.000000       ground  13.000000   3.0000   3.0000
115  Default  14.000000         long  14.000000   2.0000   2.0000
467  Default   8.000000     terminal   8.000000   1.0000   1.0000
..       ...        ...          ...        ...      ...      ...
479   Topic4   3.374766         miss   5.699818   0.8845  -6.1313
75    Topic4   7.979428    stuttgart  18.781178   0.5526  -5.2707
29    Topic4  18.042332        check  59.445744   0.2163  -4.4549
142   Topic4   3.327232         fast   5.684859   0.8729  -6.1455
66    Topic4  11.270895     friendly  31.108503   0.3934  -4.9254
141   Topic4   8.047487         fare  19.647039   0.5160  -5.2622
14    Topic4  20.807562         seat  78.612984   0.0794  -4.3123
180   Topic4  11.587855        plane  34.233738   0.3254  -4.8976
74    Topic4  13.466928        staff  42.957962   0.2486  -4.7474
39    Topic4  11.124519         late  33.275097   0.3130  -4.9384
165   Topic4   7.815802  comfortable  19.557991   0.4914  -5.2915
53    Topic4  18.083000         time  75.515335  -0.0208  -4.4526
22    Topic4  11.126114      airline  40.172890   0.1247  -4.9383
355   Topic4   5.698888      arrival  12.996371   0.5842  -5.6073
69    Topic4   7.205435     sandwich  19.865629   0.3945  -5.3728
3     Topic4  12.014500        delay  51.774296  -0.0522  -4.8615
97    Topic4   8.177223      airport  25.898882   0.2558  -5.2462
127   Topic4  10.859868       return  49.513546  -0.1086  -4.9625
2     Topic4  10.307549         crew  45.806393  -0.0829  -5.0147
123   Topic4   9.113071        price  36.905025   0.0100  -5.1379
281   Topic4   6.849676    efficient  19.126234   0.3818  -5.4234
208   Topic4   9.486576          pay  42.715023  -0.0961  -5.0977
30    Topic4   9.978504      cologne  50.279438  -0.2086  -5.0472
11 

In [38]:
# view the titles
vis_titles = pyLDAvis.gensim.prepare(lda_title, doc_term_mat_title, dct_title, sort_topics=False)
vis_titles

C:\Users\evany\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.175662 -0.043899       1        1  23.215034
1      0.228842 -0.019065       2        1  70.150467
2     -0.053179  0.062964       3        1   6.634500, topic_info=   Category        Freq          Term       Total  loglift  logprob
44  Default  107.000000      customer  107.000000  30.0000  30.0000
8   Default    6.000000      friendly    6.000000  29.0000  29.0000
9   Default    5.000000         staff    5.000000  28.0000  28.0000
3   Default    3.000000          crew    3.000000  27.0000  27.0000
25  Default    2.000000         cabin    2.000000  26.0000  26.0000
18  Default    2.000000        little    2.000000  25.0000  25.0000
4   Default    1.000000          good    1.000000  24.0000  24.0000
16  Default    1.000000         money    1.000000  23.0000  23.0000
20  Default    2.000000  professional    2.000000  22.0000  22.0000
31  Default    2.000000         delay    2.000000  21.0000  21.0000
34  Default    1.000000          late    1.000000  20.0000  20.0000
33  Default    1.000000           don    1.000000  19.0000  19.0000
19  Default    1.000000           pay    1.000000  18.0000  18.0000
35  Default    1.000000          book    1.000000  17.0000  17.0000
27  Default    1.000000       impress    1.000000  16.0000  16.0000
36  Default    1.000000           vow    1.000000  15.0000  15.0000
5   Default    1.000000         smile    1.000000  14.0000  14.0000
32  Default    1.000000       problem    1.000000  13.0000  13.0000
15  Default    1.000000         great    1.000000  12.0000  12.0000
17  Default    1.000000         value    1.000000  11.0000  11.0000
11  Default    1.000000         treat    1.000000  10.0000  10.0000
10  Default    1.000000         badly    1.000000   9.0000   9.0000
42  Default    1.000000      heathrow    1.000000   8.0000   8.0000
1   Default    1.000000       legroom    1.000000   7.0000   7.0000
43  Default    1.000000        minute    1.000000   6.0000   6.0000
30  Default    1.000000       service    1.000000   5.0000   5.0000
38  Default    1.000000      terrible    1.000000   4.0000   4.0000
0   Default    1.000000          fine    1.000000   3.0000   3.0000
12  Default    1.000000         cramp    1.000000   2.0000   2.0000
23  Default    1.000000     satisfied    1.000000   1.0000   1.0000
..      ...         ...           ...         ...      ...      ...
32   Topic3    0.259597       problem    1.328181   1.0805  -3.8561
3    Topic3    0.543568          crew    3.320812   0.9031  -3.1171
31   Topic3    0.201635         delay    2.117478   0.3614  -4.1088
17   Topic3    0.135893         value    1.429580   0.3596  -4.5034
15   Topic3    0.135893         great    1.429580   0.3596  -4.5034
10   Topic3    0.135893         badly    1.429580   0.3596  -4.5034
43   Topic3    0.135893        minute    1.429580   0.3596  -4.5034
23   Topic3    0.135893     satisfied    1.429580   0.3596  -4.5034
0    Topic3    0.135893          fine    1.429580   0.3596  -4.5034
12   Topic3    0.135893         cramp    1.429580   0.3596  -4.5034
30   Topic3    0.135893       service    1.429580   0.3596  -4.5034
37   Topic3    0.135893          food    1.429580   0.3596  -4.5034
38   Topic3    0.135893      terrible    1.429580   0.3596  -4.5034
2    Topic3    0.135893          seat    1.429580   0.3596  -4.5034
1    Topic3    0.135893       legroom    1.429580   0.3596  -4.5034
42   Topic3    0.135893      heathrow    1.429580   0.3596  -4.5034
11   Topic3    0.135893         treat    1.429580   0.3596  -4.5034
24   Topic3    0.135893          info    1.527167   0.2936  -4.5034
26   Topic3    0.135893  unresponsive    1.601762   0.2459  -4.5034
29   Topic3    0.135893         water    1.601762   0.2459  -4.5034
44   Topic3    1.538150      customer  107.256310  -1.5318  -2.0769
22   Topic3    0.135893        choice    1.601763   0.2459  -4.5034
40   Topic3    

### Topic Modelling: Assigning each Review a Topic

In [60]:
foo = np.array(lda_review.get_document_topics(doc_term_mat_review))


In [62]:
for i in foo:
    print(i[0])

(0, 0.03331475)
(0, 0.042398434)
(0, 0.6363681)
(0, 0.016690385)
(0, 0.033162974)
(0, 0.962672)
(0, 0.5031845)
(0, 0.06974237)
(0, 0.01232964)
(0, 0.79538727)
(0, 0.023669265)
(0, 0.03400143)
(0, 0.059328347)
(0, 0.010926721)
(0, 0.012124454)
(0, 0.016962335)
(0, 0.016078783)
(2, 0.010099042)
(0, 0.020320816)
(0, 0.021185687)
(0, 0.01676147)
(0, 0.031297892)
(0, 0.017716292)
(1, 0.98187846)
(0, 0.02997663)
(0, 0.29826367)
(0, 0.95356214)
(0, 0.65809596)
(0, 0.0192667)
(0, 0.02204956)
(1, 0.9682424)
(3, 0.97680086)
(0, 0.014991868)
(0, 0.021932801)
(0, 0.010459492)
(1, 0.970211)
(0, 0.018222269)
(0, 0.7371355)
(0, 0.024388636)
(0, 0.011730118)
(0, 0.024468815)
(0, 0.012947903)
(0, 0.020906618)
(0, 0.010232126)
(0, 0.021686092)
(0, 0.012317878)
(0, 0.2559234)
(0, 0.9342507)
(1, 0.012624456)
(0, 0.015519729)
(0, 0.9552061)
(0, 0.010061496)
(0, 0.023017656)
(0, 0.025524076)
(0, 0.02031984)
(0, 0.01749373)
(0, 0.03064206)
(0, 0.96246916)
(1, 0.010159399)
(0, 0.01832584)
(0, 0.017950801)
(0,